# Monte Carlo Markov Chain Method
* Oscar A. Esquivel-Flores
* Óscar A. Alvarado Morán

Vamos a utilizar el paquete `LinearAlgebra` para calcular normas y obtener matrices especiales, entre otras cosas.

In [1]:
using LinearAlgebra, BenchmarkTools

Sea el sistema de ecuaciones lineales algebraico:

\begin{align}
3x -2y & = & -3 \\ 
x + 3 - y & = & -3 \\ 
2x + y + 4z & = & 6 
\end{align}

En la representación matricial del sistema $Ax = b$, la matriz $A$ corresponde a los coeficientes, el vector $x$ contiene las variables del sistema y el vector $b$ corresponde a los términos independientes.


In [2]:
A = [3 -2 0; 1 3 -1; 2 1 4]
b = [-3, -3, 6]

3-element Array{Int64,1}:
 -3
 -3
  6

In [3]:
A

3×3 Array{Int64,2}:
 3  -2   0
 1   3  -1
 2   1   4

In [4]:
b

3-element Array{Int64,1}:
 -3
 -3
  6

Usamos el operador `\` de Juia(base) para calcular la solución del sistema, dados $A$ y $b$.

In [5]:
A\b

3-element Array{Float64,1}:
 -1.0
  0.0
  2.0

La solución del sistema es $x = [-1, 0 ,2]$

El método de Monte Carlo-Cadenas de Markov inicia descomponiendo la matriz $A = M - N$ para obtener la expresión iterativa:

\begin{equation}
x_{k+1} = Tx_{k} + f,
\end{equation}

para $k = 0, 1, 2, ...,$

donde $T=M^{-1}N$ y $f = M^{-1}b$

### Contrucción de M y N

En Julia la matriz identidad se construye mediante una sintaxis un poco freaky, la ilustro abajo, la vamos a utilizar más adelante...

In [6]:
Matrix{Float64}(I, 3, 3)

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Una forma de construir $M$ es extraer la diagonal principal de $A$ y rellenar de ceros,

In [7]:
M = Diagonal(A) # Tal vez esta es una mejor forma(?)

3×3 Diagonal{Int64,Array{Int64,1}}:
 3  ⋅  ⋅
 ⋅  3  ⋅
 ⋅  ⋅  4

$N$ la obtenemos sustituyendo $M$ de $A = M-N$

In [8]:
N = M - A

3×3 Array{Int64,2}:
  0   2  0
 -1   0  1
 -2  -1  0

Obtenemos la matriz $T$ haciendo $M^{-1}N$

In [9]:
T = inv(M) * N

3×3 Array{Float64,2}:
  0.0        0.666667  0.0
 -0.333333   0.0       0.333333
 -0.5       -0.25      0.0

La norma de $T$ y el valor de sus eigenvalores nos permite conocer la convergencia del método. La norma-`p`está definida como:

$$ \|A\|_p = \left( \sum_{i=1}^n | a_i | ^p \right)^{1/p} $$

donde `p` es cualquier número entero.

La función `norm()` de Julia calcula la norma 2 de un vector...

In [10]:
norm(T)

0.9895285072531598

In [11]:
eigvals(T)

3-element Array{Complex{Float64},1}:
 -0.2865958167935963 + 0.0im
 0.14329790839679796 - 0.6059359926660953im
 0.14329790839679796 + 0.6059359926660953im

Si los eigenvalores de $T$ están dentro del círculo unitario o la norma 2 de $T$ es menor que uno, se garantiza la convergencia del método. 

Obtenemos el vector $f$ haciendo $M^{-1}b$

In [12]:
f = inv(M) * b

3-element Array{Float64,1}:
 -1.0
 -1.0
  1.5

In [13]:
nT, mT = size(T)

(3, 3)

El proceso de Markov nos va a permitir estimar cada elemento del vector solución $x$, el proceso consiste en crear una cadena de estados que pueden interpretarse como una secuencia de elecciones aleatorias de elementos de $T$ y $f$ con los cuales crear pesos (cocientes) de la forma:

\begin{equation}
W_m = W_{m-1}\cdot w_{r_{m-1}r_m}
\end{equation}
donde $w_{r_{m-1}r_m}=\dfrac{ t_{r_{m-1}r_m}} {p_{r_{m-1}r_m} }$ y $W_0 = \dfrac{h_{r_0}}{p_{r_0}}$

Para hacer la secuencia aleatoria se requiere construir una matriz de probabilidad $P$ para elegir cada elemento no cero de $T$. Iniciamos creando un vector $S$ en donde se guardan el número de elementos no cero de $T$ por cada renglón. 

Definimos la matriz $S$ donde se guardarán los valores no cero por renglón. Este vector se usa para calcular la matriz de probabilidad $P$. Usamos una _comprehension list_ de arreglos para recorrer $T$.

In [14]:
S = zeros(Int64, nT)

3-element Array{Int64,1}:
 0
 0
 0

In [15]:
[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]

5-element Array{Int64,1}:
 1
 1
 1
 2
 2

In [16]:
S

3-element Array{Int64,1}:
 1
 2
 2

Cada renglón de la matriz $P$ contiene valores de probabilidad (uniforme) para cada elemento de $T$ diferentes de cero.

In [17]:
P = zeros(nT, mT)
[P[i,j] = 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0];

In [18]:
P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.5  0.0  0.5
 0.5  0.5  0.0

El vector $P_i$ contiene la probabilidad uniforme de elegir un elemento no cero de $f$

In [19]:
Pi = [1/nT for i in 1:nT]

3-element Array{Float64,1}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

Dos parámetros nos van a permitir obtener cierta precisión en la estimación. Definimos el error de exactitud $\epsilon$ y el error estadístico $\delta$. 

In [20]:
ϵ = 0.5
δ = 0.5

0.5

Para estimar cada elemento del vector solución se requiere construir $N$ cademas de Markov y obtener el promedio. El número de cadenas de Markov $N$ se calcula haciendo:

In [21]:
N = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
N = 10

10

Para mayor exactitud en la estimación los valores de los errores $\epsilon$ y $\delta$ deberán reducirse, lo que se traduce en cadenas más grandes, resultando en el incremento del valor de $N$ y por tanto los cálculos.

El algoritmo Monte Carlo-Cadena de Markov se implementa a continuación:

In [33]:
function MCMC(N,T,f,P,ϵ)
    nT, mT = size(T)
    X = zeros(Float32, mT)
    cont1 = 0
    cont2 = 0
    lista1 = []
    for column_T in 1:nT # Separemos todo primero por columnas. Por qué por columnas(mT)?
        W_0 = 1
        R = 0
        lista = []
        for chain in 1:N # Número de Cadenas de Markov
            W = W_0
            point = column_T # ---------------------------------------------------Aquí
            X[column_T] = W_0 * f[column_T] # ---------------------------------------Aquí
            cont = 0
            while abs(W) >= ϵ
                #println(X[column_T])
                nextpoint  = 1
                u = rand()
                while u >= sum(P[point, 1:nextpoint])
                    nextpoint += 1
                    cont1 += 1
                end
                if T[point, nextpoint] != 0 # Qué pasa cuando no se aplica esto?
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])
                    #println(W_new)
                    X[column_T] += W_new * f[nextpoint] # ------------------------Aquí
                    cont2 += 1
                end
                println(W_new)
                W = W_new
                point = nextpoint
                cont += 1
            end
            println(X[column_T])
            println("------")
            R += X[column_T] # ---------------------------------------------------Aquí
            push!(lista, cont)
        end
        push!(lista1, lista)
        println(R/N)
    end
    @show cont1, cont2
    return lista1
end

MCMC (generic function with 1 method)

In [34]:
L = MCMC(7000,T,f,P,0.005)

0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.011561019943888404
-0.007707346629258935
0.0038536733146294676
-0.493701
------
0.6666666666666666
0.4444444444444444
-0.2222222222222222
-0.14814814814814814
0.07407407407407407
0.04938271604938271
-0.024691358024691357
0.016460905349794237
0.010973936899862823
0.007315957933241882
-0.007315957933241882
-0.004877305288827921
-0.9795763
------
0.6666666666666666
0.4444444444444444
-0.2222222222222222
-0.14814814814814814
0.14814814814814814
0.09876543209876543
-0.06584362139917695
-0.04389574759945129
-0.029263831732967528
0.014631915866483764
0.009754610577655842
-0.004877305288827921
-1.1761925
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.

0.07407407407407407
0.04938271604938271
-0.024691358024691357
0.016460905349794237
0.010973936899862823
0.007315957933241882
-0.007315957933241882
-0.004877305288827921
-0.9795763
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.011561019943888404
0.007707346629258935
0.005138231086172623
0.0034254873907817486
-1.4489567
------
0.6666666666666666
0.4444444444444444
-0.2222222222222222
-0.14814814814814814
0.07407407407407407
-0.04938271604938271
-0.03292181069958847
-0.021947873799725647
0.021947873799725647
0.014631915866483764
0.009754610577655842
-0.004877305288827921
-1.0417619
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.01156101994

-0.19753086419753085
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.08779149519890259
-0.058527663465935055
-0.03901844231062337
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.011561019943888404
0.007707346629258935
0.005138231086172623
-0.0034254873907817486
-1.37751
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
-0.03901844231062337
0.03901844231062337
0.026012294873748912
0.017341529915832606
-0.008670764957916303
0.005780509971944202
0.0038536733146294676
-1.2073343
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.04389574759945129
-0.029263831732967528
0.014631915866483764
0.009754610577655842
-0.009754610577655842
-0.006503073718437228
0.004335382478958151
-1.1115344
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
0.19753086419753085

-0.007315957933241882
-0.004877305288827921
-0.54336226
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
0.017341529915832606
-0.008670764957916303
-0.005780509971944202
0.005780509971944202
0.0038536733146294676
-0.35912353
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
-0.03901844231062337
0.019509221155311684
-0.013006147436874456
-0.008670764957916303
-0.005780509971944202
0.002890254985972101
-0.8063754
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.04389574759945129
-0.029263831732967528
0.029263831732967528
0.019509221155311684
-0.013006147436874456
-0.008670764957916303
0.005780509971

0.1316872427983539
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.011561019943888404
-0.007707346629258935
0.007707346629258935
0.005138231086172623
0.0034254873907817486
-1.1642214
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.04389574759945129
0.029263831732967528
0.019509221155311684
-0.013006147436874456
-0.008670764957916303
0.005780509971944202
0.0038536733146294676
-0.745921
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.08779149519890259
-0.058527663465935055
-0.03901844231062337
0.019509221155311684
-0.013006147436874456
-0.008670764957916303
-0.005780509971944202
0.002890254985972101
-1.0450586
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.098765432098765

...

-0.14814814814814814
0.07407407407407407
-0.04938271604938271
-0.03292181069958847
0.021947873799725647
0.014631915866483764
-0.009754610577655842
-0.006503073718437228
-0.004335382478958151
-1.0185946
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.011561019943888404
0.007707346629258935
0.005138231086172623
-0.0034254873907817486
-1.022686
------
0.6666666666666666
0.4444444444444444
-0.2222222222222222
0.14814814814814814
0.09876543209876543
0.06584362139917695
-0.03292181069958847
-0.021947873799725647
0.010973936899862823
0.007315957933241882
-0.003657978966620941
-0.9222678
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.04389574759945129
0.029263831732967528
0.019509221155311684
0.013006147436874456

-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
-0.017341529915832606
-0.011561019943888404
-0.007707346629258935
0.007707346629258935
0.005138231086172623
0.0034254873907817486
-0.793851
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.09876543209876543
-0.06584362139917695
-0.04389574759945129
-0.029263831732967528
0.029263831732967528
0.019509221155311684
0.013006147436874456
-0.006503073718437228
0.004335382478958151
-1.2822402
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
0.03901844231062337
0.026012294873748912
0.017341529915832606
-0.008670764957916303
0.005780509971944202
0.0038536733146294676
-1.1488066
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.19753086419753085
0.1316872427983539
0.08779149519890259
-0.04389574759945129
-0.0292638317329

Excessive output truncated after 524290 bytes.

0.013006147436874456
-0.013006147436874456
-0.008670764957916303
-0.005780509971944202
0.002890254985972101
-0.9340998
------
0.6666666666666666
0.4444444444444444
-0.2222222222222222
0.14814814814814814
0.09876543209876543
-0.06584362139917695
-0.04389574759945129
0.029263831732967528
0.019509221155311684
0.013006147436874456
-0.013006147436874456
-0.008670764957916303
-0.005780509971944202
0.005780509971944202
0.0038536733146294676
-0.94084376
------
0.6666666666666666
0.4444444444444444
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.09876543209876543
-0.06584362139917695
-0.04389574759945129
-0.029263831732967528
0.014631915866483764
0.009754610577655842
-0.009754610577655842
-0.006503073718437228
0.004335382478958151
-0.5765551
------
0.6666666666666666
-0.4444444444444444
-0.2962962962962963
-0.19753086419753085
0.19753086419753085
0.1316872427983539
-0.08779149519890259
-0.058527663465935055
-0.03901844231062337
0.019509221155311684
-0.013006147436874456
-0.008670

3-element Array{Any,1}:
 Any[14, 12, 12, 14, 16, 15, 16, 13, 15, 15  …  13, 18, 14, 13, 16, 14, 14, 13, 11, 14]
 Any[14, 13, 18, 12, 19, 14, 15, 16, 15, 16  …  17, 14, 14, 11, 12, 15, 13, 14, 15, 13]
 Any[12, 17, 14, 14, 15, 14, 12, 18, 15, 16  …  17, 13, 11, 13, 16, 14, 12, 16, 12, 14]

In [24]:
maximum.(L)

3-element Array{Int64,1}:
 34
 33
 35

In [25]:
MCMC(70534,T,f,P,ϵ)

-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666

-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-

-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.666666

-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1

-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.999999

...

-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222

-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1

Excessive output truncated after 524293 bytes.

-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.6666666
-1.2222222
-1.6666666
-0.99999994
-1.66

3-element Array{Any,1}:
 Any[2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
 Any[2, 3, 3, 2, 2, 3, 2, 3, 3, 2  …  2, 2, 3, 2, 2, 2, 2, 2, 2, 2]
 Any[2, 3, 2, 3, 3, 2, 3, 2, 2, 2  …  3, 3, 3, 2, 3, 3, 2, 3, 3, 2]

Tabla comparativa:

| $\epsilon$  | $\delta$ | no. cadenas |    Aproximación $x$    | tiempo  |
| -------- | ----------- |-------------| -----------------------| ------- |
| 0.5      | 0.5         |   70534     | [-1.1114, 0.416, 2.09] |    4.024 s    |
| 0.05     | 0.05        | 7.053354e6  | [-0.975 , 0.003, 1.978] |    412.273 s    |


Algunas preguntas surgen:
1. El algoritmo funciona para matrices que no tiene $T$ diagonal dominante, es decir que no tengan norma 2 menor que uno o los eigenvalores no estén dentro del círculo unitario?
2. Si las dimensiones del sistema aumentan, ¿sigue siendo efectivo el método?